In [2]:
import pandas as pd
import numpy as np

In [13]:
data = {"Pink Floyd":  [1,1,1,1,1,1,1,1,1,1,1,1,1],
        "The Beatles": [0,1,1,0,0,1,1,1,1,1,1,1,1],
        "R.E.M": [0,0,1,1,0,1,0,1,1,1,1,0,0],
        "Nirvana": [1,1,0,0,0,0,0,0,1,0,0,1,1],
        "Queen": [1,1,0,0,1,0,1,0,1,1,1,1,0],
        "Oasis": [1,0,1,1,0,0,1,1,0,0,1,0,0],
        "Category": ['J','J','J','J','J','J',
                     'A','A','A','A','A','A','A']}

vdataFrame = pd.DataFrame(data) 
vdataFrame

,Pink Floyd,The Beatles,R.E.M,Nirvana,Queen,Oasis,Category
0,1,0,0,1,1,1,J
1,1,1,0,1,1,0,J
2,1,1,1,0,0,1,J
3,1,0,1,0,0,1,J
4,1,0,0,0,1,0,J
5,1,1,1,0,0,0,J
6,1,1,0,0,1,1,A
7,1,1,1,0,0,1,A
8,1,1,1,1,1,0,A
9,1,1,1,0,1,0,A


In [15]:
dataFrame = pd.read_csv('/home/erick/Desktop/mcc20192/mlearning/tareas/tarea2/nb_data/problema1.csv')
dataFrame

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Pink Floyd,1,1,1,1,1,1,1,1,1,1,1,1,1
1,The Beatles,0,1,1,0,0,1,1,1,1,1,1,1,1
2,R.E.M.,0,0,1,1,0,1,0,1,1,1,1,0,0
3,Nirvana,1,1,0,0,0,0,0,0,1,0,0,1,1
4,Queen,1,1,0,0,1,0,1,0,1,1,1,1,0
5,Oasis,1,0,1,1,0,0,1,1,0,0,1,0,0
6,Category,'J','J','J','J','J','J','A','A','A','A','A','A','A'
